In [ ]:
# article_text = """Automatic text summarization has been an active field of research for many years . Several approaches have been proposed ranging from simple position and word frequency methods to learning and graph based algorithms . The advent of human generated knowledge bases like Wikipedia offer a further possibility in text summarization they can be used to understand the input text in terms of salient concepts from the knowledge base . In this paper we study a novel approach that leverages Wikipedia in conjunction with graph based ranking . Our approach is to first construct a bipartite sentence concept graph and then rank the input sentences using iterative updates on this graph . We consider several models for the bipartite graph and derive convergence properties under each model . Then we take up personalized and query focused summarization where the sentence ranks additionally depend on user interests and queries respectively . Finally we present a Wikipedia based multi document summarization algorithm . An important feature of the proposed algorithms is that they enable real time incremental summarization users can first view an initial summary and then request additional content if interested . We evaluate the performance of our proposed summarizer using the ROUGE metric and the results show that leveraging Wikipedia can significantly improve summary quality . We also present results from a user study which suggests that using incremental summarization can help in better understanding news articles ."""

In [ ]:
import re 
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
df = pd.read_csv('/content/train_ts.csv')

In [ ]:
import heapq
import csv

for index, row in df.iterrows():
  article_text = row['Abstract'].lower()
  # remove spaces, punctuations and numbers
  clean_text = re.sub('[^a-zA-Z]', ' ', article_text)
  clean_text = re.sub('\s+', ' ', clean_text)
  # split into sentence list
  sentence_list = nltk.sent_tokenize(article_text)
  # word frequencies
  stopwords = nltk.corpus.stopwords.words('english')

  word_frequencies = {}
  for word in nltk.word_tokenize(clean_text):
    if word not in stopwords:
      if word not in word_frequencies:
        word_frequencies[word] = 1
      else:
        word_frequencies[word] += 1

  # maximum frequency
  maximum_frequency = max(word_frequencies.values())

  for word in word_frequencies:
    word_frequencies[word] = word_frequencies[word] / maximum_frequency

  # sentence score
  sentence_scores = {}

  for sentence in sentence_list:
    for word in nltk.word_tokenize(sentence):
      if word in word_frequencies and len(sentence.split(' ')) < 30:
        if sentence not in sentence_scores:
          sentence_scores[sentence] = word_frequencies[word]
        else:
          sentence_scores[sentence] += word_frequencies[word]

  # get top 5 sentences
  summary = heapq.nlargest(1 , sentence_scores, key=sentence_scores.get)

  output = " ".join(summary)

  with open('output.csv', 'a') as f:
    f.write(str(index + 1) + ') ' + output + '\n')
  # print(str(index + 1) + ') ' + output + '\n')


In [56]:
pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=12328870d139fefc505348dfb4206ca81e16d18cda4c07c58d6fccd557baeef6
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [58]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
a = 'ncappc performs i NCA and ii simulation based posterior predictive checks using NCA metrics. ncappc package is highly flexible and comprehensive. Can perform both individual and population level diagnostics. Produces output summarizing the diagnostic results including the model specific outliers. The output is easy to interpret and to use in evaluation of a population PK model.'
b = 'we developed a new package in r called ncappc to perform a nca and simulation based posterior predictive checks for a population pk model using nca metrics .'
scores = scorer.score(a, b)

In [59]:
scores

{'rouge1': Score(precision=0.7037037037037037, recall=0.3392857142857143, fmeasure=0.4578313253012048),
 'rougeL': Score(precision=0.48148148148148145, recall=0.23214285714285715, fmeasure=0.3132530120481928)}